<a href="https://colab.research.google.com/github/Iagofer04/JogosCopa/blob/main/Projeto2_copa.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [89]:
import pandas as pd
import numpy as np
import random
from scipy.stats import poisson

selecoes = pd.read_excel('/content/DadosCopaDoMundoQatar2022.xlsx', sheet_name = 'selecoes', index_col = 0)
jogos = pd.read_excel('/content/DadosCopaDoMundoQatar2022.xlsx', sheet_name = 'jogos')



In [74]:

selecoes.head()


,Grupo,GrupoNumero,NomeEmIngles,PosiçãoRankingFIFA,ValorDeMercado,PontosRankingFIFA,Confederação,Copas,JogadorDestaque,FotoJogadorDestaque,LinkBandeiraPequena,LinkBandeiraGrande
Seleção,,,,,,,,,,,,
Catar,A,A1,Qatar,48,18.33,1442,AFC,0,Almoez Ali,https://github.com/ricardorocha86/PrevisaoEspo...,https://cloudinary.fifa.com/api/v3/picture/fla...,https://flagdownload.com/wp-content/uploads/Fl...
Equador,A,A2,Ecuador,44,125.80,1464,UEFA,0,Pervis Estupiñán,https://github.com/ricardorocha86/PrevisaoEspo...,https://cloudinary.fifa.com/api/v3/picture/fla...,https://flagdownload.com/wp-content/uploads/Fl...
Senegal,A,A3,Senegal,18,286.50,1585,CAF,0,Sadio Mané,https://github.com/ricardorocha86/PrevisaoEspo...,https://cloudinary.fifa.com/api/v3/picture/fla...,https://flagdownload.com/wp-content/uploads/Fl...
Holanda,A,A4,Netherlands,8,455.75,1679,UEFA,0,Virgil Van Dijk,https://github.com/ricardorocha86/PrevisaoEspo...,https://cloudinary.fifa.com/api/v3/picture/fla...,https://flagdownload.com/wp-content/uploads/Fl...
Inglaterra,B,B1,England,5,1360.00,1737,UEFA,1,Harry Kane,https://github.com/ricardorocha86/PrevisaoEspo...,https://cloudinary.fifa.com/api/v3/picture/fla...,https://flagdownload.com/wp-content/uploads/Fl...


In [75]:
jogos.head()


,data,hora,grupo,seleção1,seleção2,estádio
0,2022-11-21,07:00:00,A,Senegal,Holanda,AL THUMAMA
1,2022-11-21,13:00:00,A,Catar,Equador,AL KHOR
2,2022-11-25,10:00:00,A,Catar,Senegal,AL THUMAMA
3,2022-11-25,13:00:00,A,Holanda,Equador,INTER. KHALIFA
4,2022-11-29,12:00:00,A,Holanda,Catar,AL KHOR


In [76]:
fifa = selecoes ['PontosRankingFIFA']
a, b = min(fifa), max(fifa)
fa, fb = 0.15, 1
b1 = (fb - fa)/(b-a)
b0 = fb - b*b1
forca = b0 + b1*fifa
forca

Seleção
Catar             0.243596
Equador           0.285618
Senegal           0.516742
Holanda           0.696292
Inglaterra        0.807079
Irã               0.467079
Estados Unidos    0.612247
País de Gales     0.511011
Argentina         0.872022
Arábia Saudita    0.232135
México            0.640899
Polônia           0.442247
França            0.860562
Dinamarca         0.669551
Tunísia           0.369663
Austrália         0.323820
Espanha           0.768876
Alemanha          0.658090
Japão             0.459438
Costa Rica        0.354382
Bélgica           0.969438
Canadá            0.304719
Marrocos          0.465169
Croácia           0.606517
Brasil            1.000000
Sérvia            0.449888
Suíça             0.585506
Camarões          0.325730
Portugal          0.696292
Gana              0.150000
Uruguai           0.623708
Coreia do Sul     0.404045
Name: PontosRankingFIFA, dtype: float64

In [77]:
def MediasPoisson(selecao1, selecao2):
  forca1 = forca[selecao1]
  forca2 = forca[selecao2]
  mgols = 2.75
  l1 = mgols*forca1/(forca1 + forca2)
  l2 = mgols - l1
  return [l1, l2]

def Resultado(gols1, gols2):
  if gols1 > gols2:
    resultado = 'V'
  elif gols2 > gols1:
    resultado = 'D'
  else:
    resultado = 'E'
  return resultado         

def Pontos(gols1, gols2):
  resultado = Resultado(gols1, gols2)
  if resultado == 'V':
    pontos1, pontos2 = [3, 0]
  elif resultado == 'D':
    pontos1, pontos2 = [0, 3]
  else:
    pontos1, pontos2 = [1, 1]    
  return [pontos1, pontos2, resultado]
    

In [78]:
def Jogo(selecao1, selecao2):
  l1, l2 = MediasPoisson(selecao1, selecao2)
  gols1 = int(np.random.poisson(lam  = l1, size = 1))
  gols2 = int(np.random.poisson(lam  = l2, size = 1))
  saldo1 = gols1 - gols2
  saldo2 = -saldo1
  pontos1, pontos2, resultado = Pontos(gols1, gols2)
  placar = '{} x {}'.format(gols1, gols2)
  return [gols1, gols2, saldo1, saldo2, pontos1, pontos2, resultado, placar]


In [79]:
def Distribuicao(media):
  probs = []
  for i in range (7):
      probs.append(poisson.pmf (i, media))
  probs.append (1 - sum(probs))
  return pd.Series(probs, index = ['0', '1', '2', '3', '4', ' 5', '6', '7+' ]) 

def ProbabilidadesPartidas(selecao1, selecao2):
    l1, l2 = MediasPoisson(selecao1, selecao2)
    d1, d2 = Distribuicao(l1), Distribuicao(l2)
    matriz = np.outer(d1, d2)
    vitoria = np.tril(matriz).sum() - np.trace(matriz)
    derrota = np.triu(matriz).sum() - np.trace(matriz)
    empate = 1 - (vitoria + derrota)
   
    probs = np.around ([vitoria, empate, derrota], 3)
    probsp = [f'{100*i:.1f}%' for i in probs]
    
    nomes = ['0', '1', '2', '3', '4', ' 5', '6', '7+' ]
    matriz = pd.DataFrame(matriz, columns = nomes, index = nomes)
    matriz.index = pd.MultiIndex.from_product([[selecao1], matriz.index])
    matriz.columns = pd.MultiIndex.from_product([[selecao2], matriz.columns])
    output = {'seleção1': selecao1, 'seleção2': selecao2,
              'f1': forca[selecao1], 'f2': forca [selecao2],
              'media1': l1, 'media2': l2,
              'probabilidades': probsp, 'matriz': matriz}
    return output


In [80]:
ProbabilidadesPartidas('Brasil', 'Camarões')

{'seleção1': 'Brasil',
 'seleção2': 'Camarões',
 'f1': 1.0,
 'f2': 0.32573033707865173,
 'media1': 2.0743283329095688,
 'media2': 0.6756716670904312,
 'probabilidades': ['70.0%', '19.0%', '11.0%'],
 'matriz':            Camarões                                                    \
                   0         1         2         3         4         5   
 Brasil 0   0.063928  0.043194  0.014593  0.003287  0.000555  0.000075   
        1   0.132607  0.089599  0.030270  0.006817  0.001152  0.000156   
        2   0.137536  0.092929  0.031395  0.007071  0.001194  0.000161   
        3   0.095098  0.064255  0.021708  0.004889  0.000826  0.000112   
        4   0.049316  0.033322  0.011257  0.002535  0.000428  0.000058   
         5  0.020460  0.013824  0.004670  0.001052  0.000178  0.000024   
        6   0.007073  0.004779  0.001615  0.000364  0.000061  0.000008   
        7+  0.002797  0.001890  0.000638  0.000144  0.000024  0.000003   
 
                                        
         

In [81]:
jogos['Vitória'] = None
jogos['Empate'] = None
jogos['Derrota'] = None

for i in range (jogos.shape[0]):
  selecao1 = jogos ['seleção1'][i]
  selecao2 = jogos ['seleção2'][i]
  v, e, d = ProbabilidadesPartidas(selecao1, selecao2)['probabilidades']
  jogos.at[i, 'Vitória'] = v
  jogos.at[i, 'Empate'] = e
  jogos.at[i, 'Derrota'] = d

jogos.to_excel('outputEstimativasJogosCopa.xls')
jogos


<ipython-input-81-0697f8f6ecf5>:13: FutureWarning: As the xlwt package is no longer maintained, the xlwt engine will be removed in a future version of pandas. This is the only engine in pandas that supports writing in the xls format. Install openpyxl and write to an xlsx file instead. You can set the option io.excel.xls.writer to 'xlwt' to silence this warning. While this option is deprecated and will also raise a warning, it can be globally set and the warning suppressed.
  jogos.to_excel('outputEstimativasJogosCopa.xls')


,data,hora,grupo,seleção1,seleção2,estádio,Vitória,Empate,Derrota
0,2022-11-21,07:00:00,A,Senegal,Holanda,AL THUMAMA,28.3%,24.9%,46.8%
1,2022-11-21,13:00:00,A,Catar,Equador,AL KHOR,32.3%,25.4%,42.3%
2,2022-11-25,10:00:00,A,Catar,Senegal,AL THUMAMA,17.2%,22.1%,60.6%
3,2022-11-25,13:00:00,A,Holanda,Equador,INTER. KHALIFA,64.4%,21.0%,14.6%
4,2022-11-29,12:00:00,A,Holanda,Catar,AL KHOR,68.4%,19.6%,12.0%
5,2022-11-29,12:00:00,A,Equador,Senegal,INTER. KHALIFA,20.7%,23.3%,56.0%
6,2022-11-21,10:00:00,B,Inglaterra,Irã,INTER. KHALIFA,54.6%,23.6%,21.8%
7,2022-11-21,16:00:00,B,Estados Unidos,País de Gales,AL RAYYAN,43.0%,25.3%,31.7%
8,2022-11-25,07:00:00,B,País de Gales,Irã,AL RAYYAN,40.1%,25.5%,34.4%
9,2022-11-25,16:00:00,B,Inglaterra,Estados Unidos,AL KHOR,46.1%,25.0%,28.9%


Simulação da Copa

Primeira fase


In [82]:
def JogosGrupo(dados, grupo):
  selecoes = list(dados[dados['Grupo'] == grupo].index)
  selecao1, selecao2, selecao3, selecao4 = selecoes

  jogo1 = Jogo(selecao1, selecao2)
  jogo2 = Jogo(selecao3, selecao4)
  jogo3 = Jogo(selecao1, selecao3)
  jogo4 = Jogo(selecao2, selecao4)
  jogo5 = Jogo(selecao1, selecao4)
  jogo6 = Jogo(selecao2, selecao3)


  pt1, pt2, pt3, pt4 = 0, 0, 0, 0
  gp1, gp2, gp3, gp4 = 0, 0, 0, 0
  sg1, sg2, sg3, sg4 = 0, 0, 0, 0

  gp1, gp2, sg1, sg2, pt1, pt2 = gp1 + jogo1[0], gp2 + jogo1[1], sg1 + jogo1[2], sg2 + jogo1[3], pt1 + jogo1[4], pt2 + jogo1[5]
  gp3, gp4, sg3, sg4, pt3, pt4 = gp3 + jogo2[0], gp4 + jogo2[1], sg3 + jogo2[2], sg4 + jogo2[3], pt3 + jogo2[4], pt4 + jogo2[5]
  gp1, gp3, sg1, sg3, pt1, pt3 = gp1 + jogo3[0], gp3 + jogo3[1], sg1 + jogo3[2], sg3 + jogo3[3], pt1 + jogo3[4], pt3 + jogo3[5]
  gp2, gp4, sg2, sg4, pt2, pt4 = gp2 + jogo4[0], gp4 + jogo4[1], sg2 + jogo4[2], sg4 + jogo4[3], pt2 + jogo4[4], pt4 + jogo4[5]
  gp1, gp4, sg1, sg4, pt1, pt4 = gp1 + jogo5[0], gp4 + jogo5[1], sg1 + jogo5[2], sg4 + jogo5[3], pt1 + jogo5[4], pt4 + jogo5[5]
  gp2, gp3, sg2, sg3, pt2, pt3 = gp2 + jogo6[0], gp3 + jogo6[1], sg2 + jogo6[2], sg3 + jogo6[3], pt2 + jogo6[4], pt3 + jogo6[5]

  tab = pd.DataFrame([[pt1, pt2, pt3, pt4], [sg1, sg2, sg3, sg4], [gp1, gp2, gp3, gp4]], columns = selecoes, index = ['Pontos', 'SG', 'GP']).T
  tab = tab.sort_values(by = ['Pontos', 'SG', 'GP'], ascending = False)
  tab['Posição'] = [1, 2, 3, 4]
  
  partidas = [ selecao1 + 'x' + selecao2, 
			      selecao3 + 'x' + selecao4,
			      selecao1 + 'x' + selecao3,
			      selecao2 + 'x' + selecao4,
			      selecao1 + 'x' + selecao4,
			      selecao2 + 'x' + selecao3	]

  resultados = [jogo1[6], jogo2[6], jogo3[6], jogo4[6], jogo5[6], jogo6[6]]
  placares = [jogo1[-1], jogo2[-1], jogo3[-1], jogo4[-1], jogo5[-1], jogo6[-1]]

  jogos = pd.DataFrame([partidas, placares, resultados]).transpose()
  jogos.columns = ['Partida', 'Placar', 'Resultado']
  return [tab, jogos] 



In [83]:
grupo = JogosGrupo(selecoes, 'G')
grupo[0]


,Pontos,SG,GP,Posição
Brasil,6,6,8,1
Sérvia,6,-2,3,2
Suíça,4,1,3,3
Camarões,1,-5,0,4


In [84]:
grupo[1]

,Partida,Placar,Resultado
0,BrasilxSérvia,4 x 0,V
1,SuíçaxCamarões,0 x 0,E
2,BrasilxSuíça,0 x 2,D
3,SérviaxCamarões,1 x 0,V
4,BrasilxCamarões,4 x 0,V
5,SérviaxSuíça,2 x 1,V


Seguda fase da competição

In [97]:
def JogoMataMata(selecao1, selecao2):
	jogo = Jogo(selecao1, selecao2)
	resultado = jogo[6]
	if resultado == 'V':
		return selecao1
	elif resultado == 'D':
		return selecao2
	else:
		return random.sample([selecao1, selecao2], 1)[0]



In [104]:
JogoMataMata('Brasil', 'Coreia do Sul')

'Brasil'

In [109]:
 def SimulaCopa(selecoes):
  cols = ['1st', '2sd', '3rd', '4th', 'Oitavas', 'Quartas', 'Semis', 'Final', 'Campeão']
  n = selecoes.shape[0]
  m = len(cols)
  aux = np.array(np.zeros(n*m).reshape(n,m))
  info = pd.DataFrame(aux, columns=cols, index=selecoes.index)
  info = info.astype(int)


  top16 = []
  for i in list('ABCDEFGH'):
    a = JogosGrupo(selecoes, grupo = i)[0]
    top16 += a.index[:2].tolist()
    anomes = a.index.tolist()
    info.at[ anomes[0], '1st'] = 1
    info.at[ anomes[1], '2sd'] = 1
    info.at[ anomes[2], '3rd'] = 1
    info.at[ anomes[3], '4th'] = 1

  qf1 = JogoMataMata(top16[0], top16[3])     #1A x 2B
  qf2 = JogoMataMata(top16[2], top16[1])     #1B x 2A 
  qf3 = JogoMataMata(top16[4], top16[7])     #1C x 2D
  qf4 = JogoMataMata(top16[6], top16[5])     #1D x 2C 
  qf5 = JogoMataMata(top16[8], top16[11])    #1E x 2F
  qf6 = JogoMataMata(top16[10], top16[9])    #1F x 2E
  qf7 = JogoMataMata(top16[12], top16[15])   #1G x 2H
  qf8 = JogoMataMata(top16[14], top16[13])   #1H x 2G


  top8 = [qf1, qf2, qf3, qf4, qf5, qf6, qf7, qf8]

  sf1 = JogoMataMata(qf1, qf3)     
  sf2 = JogoMataMata(qf2, qf4)      
  sf3 = JogoMataMata(qf5, qf7)     
  sf4 = JogoMataMata(qf6, qf8)      


  top4 = [sf1, sf2, sf3, sf4]

  f1 = JogoMataMata(sf1, sf3)     
  f2 = JogoMataMata(sf2, sf4) 

  top2 = [f1, f2]

  top1 = JogoMataMata(f1, f2)

  info.at[top16, 'Oitavas'] = 1
  info.at[top8, 'Quartas'] = 1
  info.at[top4, 'Semis'] = 1
  info.at[top2, 'Final'] = 1
  info.at[top1, 'Campeão'] = 1

  return info



In [121]:
def SimulacaoTotal(dados, S = 1000):
	print('IA: "Iniciando Simulação..."')
	info = SimulaCopa(dados)
	for i in range(S-1):
		info += SimulaCopa(dados)
		if (i+2)%(S/10) == 0:
			print('IA: "Simulações de Copas do Mundo: {:.0f}% completas'.format(100*((i+2)/S)))
	print('IA: "Simulação Finalizada!"')
	return info.sort_values(by = 'Campeão', ascending = False)/S



In [123]:
S = 10000
sim = SimulacaoTotal(selecoes, S)
sim.to_excel('outputSimulacoesCopaDoMundo.xlsx')
sim

IA: "Iniciando Simulação..."
IA: "Simulações de Copas do Mundo: 10% completas
IA: "Simulações de Copas do Mundo: 20% completas
IA: "Simulações de Copas do Mundo: 30% completas
IA: "Simulações de Copas do Mundo: 40% completas
IA: "Simulações de Copas do Mundo: 50% completas
IA: "Simulações de Copas do Mundo: 60% completas
IA: "Simulações de Copas do Mundo: 70% completas
IA: "Simulações de Copas do Mundo: 80% completas
IA: "Simulações de Copas do Mundo: 90% completas
IA: "Simulações de Copas do Mundo: 100% completas
IA: "Simulação Finalizada!"


,1st,2sd,3rd,4th,Oitavas,Quartas,Semis,Final,Campeão
Seleção,,,,,,,,,
Brasil,0.5916,0.2520,0.1187,0.0377,0.8436,0.5655,0.3535,0.2211,0.1351
Bélgica,0.5730,0.2589,0.1228,0.0453,0.8319,0.5392,0.3283,0.2012,0.1241
França,0.5156,0.2947,0.1376,0.0521,0.8103,0.4827,0.2968,0.1657,0.0919
Argentina,0.5362,0.2865,0.1392,0.0381,0.8227,0.4953,0.3022,0.1676,0.0904
Inglaterra,0.4361,0.2774,0.1760,0.1105,0.7135,0.4601,0.2548,0.1359,0.0721
Espanha,0.4427,0.2867,0.1802,0.0904,0.7294,0.3994,0.2096,0.1109,0.0530
Portugal,0.4554,0.3349,0.1745,0.0352,0.7903,0.4170,0.2115,0.1102,0.0519
Holanda,0.5361,0.2890,0.1277,0.0472,0.8251,0.4430,0.2177,0.1113,0.0516
Dinamarca,0.3339,0.3562,0.2092,0.1007,0.6901,0.3461,0.1832,0.0876,0.0423


In [125]:
etapas = pd.DataFrame()
etapas['Cair na 1ª Fase'] = 1 - sim['Oitavas']
etapas['Cair nas Oitavas'] = sim['Oitavas'] - sim['Quartas']
etapas['Cair nas Quartas'] = sim['Quartas'] - sim['Semis']
etapas['Cair nas Semis'] = sim['Semis'] - sim['Final']
etapas['Cair na Final'] = sim['Final'] - sim['Campeão']
etapas['Ganhar a Final'] = sim['Campeão']
etapas.to_excel('outputProbabilidadesPorEtapa.xlsx')
etapas

,Cair na 1ª Fase,Cair nas Oitavas,Cair nas Quartas,Cair nas Semis,Cair na Final,Ganhar a Final
Seleção,,,,,,
Brasil,0.1564,0.2781,0.2120,0.1324,0.0860,0.1351
Bélgica,0.1681,0.2927,0.2109,0.1271,0.0771,0.1241
França,0.1897,0.3276,0.1859,0.1311,0.0738,0.0919
Argentina,0.1773,0.3274,0.1931,0.1346,0.0772,0.0904
Inglaterra,0.2865,0.2534,0.2053,0.1189,0.0638,0.0721
Espanha,0.2706,0.3300,0.1898,0.0987,0.0579,0.0530
Portugal,0.2097,0.3733,0.2055,0.1013,0.0583,0.0519
Holanda,0.1749,0.3821,0.2253,0.1064,0.0597,0.0516
Dinamarca,0.3099,0.3440,0.1629,0.0956,0.0453,0.0423


In [126]:
avanco = pd.DataFrame()
avanco['Avançar na 1ª Fase'] = sim['Oitavas']
avanco['Avançar nas Oitavas'] = sim['Quartas']/sim['Oitavas']
avanco['Avançar nas Quartas'] = sim['Semis']/sim['Quartas']
avanco['Avançar nas Semis'] = sim['Final']/sim['Semis']
avanco['Avançar na Final'] = sim['Campeão']/sim['Final']
avanco.to_excel('outputAvancoPorEtapa.xlsx')
avanco






,Avançar na 1ª Fase,Avançar nas Oitavas,Avançar nas Quartas,Avançar nas Semis,Avançar na Final
Seleção,,,,,
Brasil,0.8436,0.670341,0.625111,0.625460,0.611036
Bélgica,0.8319,0.648155,0.608865,0.612854,0.616799
França,0.8103,0.595705,0.614875,0.558288,0.554617
Argentina,0.8227,0.602042,0.610135,0.554600,0.539379
Inglaterra,0.7135,0.644849,0.553793,0.533359,0.530537
Espanha,0.7294,0.547573,0.524787,0.529103,0.477908
Portugal,0.7903,0.527648,0.507194,0.521040,0.470962
Holanda,0.8251,0.536905,0.491422,0.511254,0.463612
Dinamarca,0.6901,0.501522,0.529327,0.478166,0.482877
